In [1]:
# Import TensorFlow library
import tensorflow as tf

# Import ImageDataGenerator class from Keras for image preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import ResNet50 architecture from Keras applications
from tensorflow.keras.applications import ResNet50

# Import Dense and GlobalAveragePooling2D layers from Keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Import Model class from Keras for defining custom models
from tensorflow.keras.models import Model


In [4]:
train_dir = '../data/train/'
test_dir = '../data/test/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)



Found 1476 images belonging to 2 classes.
Found 1776 images belonging to 2 classes.


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)



In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [7]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)


Epoch 1/3


C:\Users\masub\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


47/47 ━━━━━━━━━━━━━━━━━━━━ 572s 11s/step - accuracy: 0.8665 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 1237347.3750
Epoch 2/3


C:\Users\masub\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/3
47/47 ━━━━━━━━━━━━━━━━━━━━ 552s 12s/step - accuracy: 0.9731 - loss: 0.0932 - val_accuracy: 0.5000 - val_loss: 142.4934


In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess an image
img_path = '../data/myimage/image_1.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.

# Predict
prediction = model.predict(x)

# Visualize result
if prediction < 0.5:
    print("The image is predicted to be normal.")
else:
    print("The image is predicted to have cataract.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The image is predicted to be normal.
